In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("MLAnalysisAdClicks").getOrCreate()
sc = spark.sparkContext

In [ ]:
# Load normalized data
df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/yl9709/combined_training_before_normalize.txt")

# Select necessary columns and cast them as needed
from pyspark.sql.functions import col

df = df.select(
    col("numClick").cast("int"),
    col("region").cast("int"),
    col("city").cast("int"),
    col("adExchange").cast("int"),
    col("width").cast("int"),
    col("height").cast("int"),
    col("floorPrice").cast("int"),
    col("weekday"),
    col("hour").cast("int"),
    col("payingPrice").cast("int")
)

# Check the data
df.show(5)

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# Indexing and encoding categorical columns
categoricalColumns = ['region', 'city', 'adExchange', 'weekday']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

# Assembling vector
assemblerInputs = [c + "classVec" for c in categoricalColumns] + ['width', 'height', 'floorPrice', 'hour']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Set up the evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="numClick", metricName="areaUnderROC")

train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)


In [ ]:

# Logistic Regression
lr = LogisticRegression(labelCol="numClick", featuresCol="features", maxIter=10)
pipeline_lr = Pipeline(stages=stages + [lr])
model_lr = pipeline_lr.fit(train_df)
predictions_lr = model_lr.transform(test_df)
print("Logistic Regression AUC: ", evaluator.evaluate(predictions_lr))

In [ ]:
# Gradient Boosting Trees
gbt = GBTClassifier(labelCol="numClick", featuresCol="features", maxIter=10)
pipeline_gbt = Pipeline(stages=stages + [gbt])
model_gbt = pipeline_gbt.fit(train_df)
predictions_gbt = model_gbt.transform(test_df)
print("Gradient Boosting AUC: ", evaluator.evaluate(predictions_gbt))

In [ ]:
# Support Vector Machine
svm = LinearSVC(labelCol="numClick", featuresCol="features", maxIter=10)
pipeline_svm = Pipeline(stages=stages + [svm])
model_svm = pipeline_svm.fit(train_df)
predictions_svm = model_svm.transform(test_df)
print("SVM AUC: ", evaluator.evaluate(predictions_svm))

In [ ]:
# Add the prediction column to the test data
test_df= test_df.join(predictions_gbt.select('prediction', 'features'), on='features', how='inner')

In [ ]:
# Sum bidding price to determine the budget.
# The max bidding in the simulation will be less than this.
print(test_df['payingPrice'].sum())

In [ ]:
# (avg winning price / average click through rate) to determine the range of the alpha
print(test_df['payingPrice'].avg() / test_df['clickRate'].avg())
print(test_df['payingPrice'].max() / test_df['clickRate'].min())

# bid price = click through rate * alpha

Define the Bidding Function: 
<br>(1) Bidding Price = alpha × Predicted Click Rate
<br>(2) Implement Binary Search: Use binary search to find the optimal β value.
<br>(3) Simulation Function: Create a function to simulate bidding based on a given β and calculate the total clicks won and the total cost.
<br>(4) Optimization Loop: Use the binary search to adjust β to maximize clicks while staying under the budget.

In [ ]:
# simulate bidding to based on a given alpha and calculate the total clicks and total costs
def simulateBidding(alpha, test_df, budget):
    total_spent = 0
    total_clicks = 0

    for row in test_df.collect():
        predicted_click_rate = row.prediction
        winning_price = row.payingPrice
        bid_price = alpha * predicted_click_rate + 
        if bid_price > winning_price # and total_spent + bid_price <= budget:
            total_spent += bid_price # assumes that I am paying my bid_price (although it's a second price auction)
            total_clicks =  total_clicks + row[is_click] ## 

    return total_clicks, total_spent


# Use binary search to find the alpha that maximizes the click rate while staying under the budget
def binarySearch(low, high, test_df, budget):
    # returns the alpha
    alpha = 0
    max_clicks = 0

    while high - low > 0.01:
        mid = (low + high) / 2
        clicks, spend = simulateBidding(mid, test_df, budget)
        if spend > budget:
            high = mid
        else:
            low = mid
            if clicks > max_clicks:
                max_clicks = clicks
                alpha = mid

    return alpha

    

In [ ]:

alpha = binarySearch(0, 100, test_df, 1000) # the low & high & budget are to be changed later. 
print(alpha)